In [1]:
!pip install pandas
!pip install jiwer
!pip install gradio
!pip install python-dotenv
!pip install hopsworks

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 12.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.2/14.2 MB 90.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 KB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 64.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 KB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.5/270.5 KB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [3]:
from datetime import datetime
import time 
import requests
import sys
import pandas as pd
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
sys.path.append('/content/drive/MyDrive/ID2223-project/air quality prediction2')
from functions import *

Mounted at /content/drive


In [4]:
df_air_quality = pd.read_csv('https://raw.githubusercontent.com/YuhangDeng123/AIR-Forecast/main/GuangZhou-air-quality%20.csv')
df_air_quality.head()

,date,pm25,pm10,o3,no2,so2,co
0,2021-01-01,60,42,,25,4,4
1,2021-01-02,84,67,,35,5,6
2,2021-01-03,121,58,,27,5,5
3,2021-01-04,103,64,,28,6,6
4,2021-01-05,122,65,,23,5,5


In [5]:
df_air_quality.date = df_air_quality.date.apply(timestamp_2_time)
df_air_quality

,date,pm25,pm10,o3,no2,so2,co
0,1609459200000,60,42,,25,4,4
1,1609545600000,84,67,,35,5,6
2,1609632000000,121,58,,27,5,5
3,1609718400000,103,64,,28,6,6
4,1609804800000,122,65,,23,5,5
...,...,...,...,...,...,...,...
360,1672099200000,90,61,40,11,4,6
361,1672185600000,125,58,40,5,3,6
362,1672272000000,133,44,35,6,2,6
363,1672358400000,99,31,30,7,1,5


In [6]:
df_air_quality = df_air_quality.drop(labels=['pm10','o3','no2','so2','co'], axis=1)
df_air_quality

,date,pm25
0,1609459200000,60
1,1609545600000,84
2,1609632000000,121
3,1609718400000,103
4,1609804800000,122
...,...,...
360,1672099200000,90
361,1672185600000,125
362,1672272000000,133
363,1672358400000,99


In [8]:
df_weather = pd.read_csv('https://raw.githubusercontent.com/YuhangDeng123/AIR-Forecast/main/GuangZhou-weather.csv')
df_weather.head()

,name,date,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,solarenergy,uvindex,severerisk,sunrise,sunset,moonphase,conditions,description,icon,stations
0,guangzhou,2021-01-01,15.0,3.8,9.1,15.0,0.6,7.8,-10.9,24.6,...,17.7,8,NaN,2021-01-01T07:08:23,2021-01-01T17:52:49,0.53,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"ZGGG,59287099999,59278099999"
1,guangzhou,2021-01-02,18.0,4.8,11.0,18.0,3.2,10.8,-4.1,39.1,...,17.7,8,NaN,2021-01-02T07:08:41,2021-01-02T17:53:28,0.56,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"ZGGG,59287099999,59278099999"
2,guangzhou,2021-01-03,22.0,5.8,14.2,22.0,4.7,13.8,0.0,42.1,...,15.6,7,NaN,2021-01-03T07:08:57,2021-01-03T17:54:07,0.60,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"ZGGG,59287099999,59278099999"
3,guangzhou,2021-01-04,21.0,14.3,17.1,21.0,14.3,17.1,0.6,33.2,...,15.5,7,NaN,2021-01-04T07:09:12,2021-01-04T17:54:47,0.65,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"ZGGG,59287099999,59278099999"
4,guangzhou,2021-01-05,23.0,15.1,18.3,23.0,15.1,18.3,3.3,37.5,...,15.5,7,NaN,2021-01-05T07:09:26,2021-01-05T17:55:28,0.70,Partially cloudy,Clearing in the afternoon.,partly-cloudy-day,"ZGGG,59287099999,59278099999"


In [ ]:
df_weather = df_weather.drop(labels=['stations','icon','description','conditions','sunset','sunrise','severerisk','preciptype','name'], axis=1)
df_weather.date = df_weather.date.apply(timestamp_2_time)
df_weather

In [ ]:
import hopsworks
project = hopsworks.login()
fs = project.get_feature_store() 

In [ ]:
air_quality_fg = fs.get_or_create_feature_group(
        name = 'air_quality_fg',
        description = 'Air Quality characteristics of each day',
        version = 1,
        primary_key = ['date'],
        online_enabled = True,
        event_time = 'date'
    )    

air_quality_fg.insert(df_air_quality)

weather_fg = fs.get_or_create_feature_group(
        name = 'weather_fg',
        description = 'Weather characteristics of each day',
        version = 1,
        primary_key = ['date'],
        online_enabled = True,
        event_time = 'date'
    )    

weather_fg.insert(df_weather)

(<hsfs.core.job.Job at 0x7f2019c927f0>, None)

In [ ]:
# The following is to do the data cleaning
df_air_quality = df_air_quality.drop(labels=['date'], axis=1)
sum_updata = pd.concat([df_air_quality,df_weather],axis=1)
sum_updata

,pm25,date,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,windgust,windspeed,winddir,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,moonphase
0,60,1609459200000,15.0,3.8,9.1,15.0,0.6,7.8,-10.9,24.6,...,29.7,21.6,0.7,1027.6,47.1,13.5,205.4,17.7,8,0.53
1,84,1609545600000,18.0,4.8,11.0,18.0,3.2,10.8,-4.1,39.1,...,25.2,8.6,34.9,1023.8,20.6,13.2,205.4,17.7,8,0.56
2,121,1609632000000,22.0,5.8,14.2,22.0,4.7,13.8,0.0,42.1,...,31.6,18.0,7.9,1023.1,40.4,12.9,179.5,15.6,7,0.60
3,103,1609718400000,21.0,14.3,17.1,21.0,14.3,17.1,0.6,33.2,...,22.1,18.0,357.3,1022.0,60.0,13.5,179.4,15.5,7,0.65
4,122,1609804800000,23.0,15.1,18.3,23.0,15.1,18.3,3.3,37.5,...,25.0,21.6,358.0,1021.0,44.1,13.4,178.6,15.5,7,0.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,90,1672099200000,22.0,9.2,15.1,22.0,8.6,15.0,6.6,59.6,...,21.2,14.4,37.9,1022.7,3.4,13.3,194.0,16.8,7,0.11
361,125,1672185600000,20.0,12.6,15.6,20.0,12.6,15.6,5.6,52.0,...,35.2,28.8,356.8,1023.6,34.1,13.3,194.5,16.8,7,0.16
362,133,1672272000000,18.0,10.9,13.8,18.0,10.9,13.8,4.5,53.8,...,35.2,21.6,355.3,1026.0,49.6,13.2,176.0,15.2,7,0.22
363,99,1672358400000,17.0,8.0,11.9,17.0,5.5,11.2,4.9,63.3,...,36.0,25.2,359.1,1027.1,40.8,13.2,191.5,16.6,7,0.27


In [ ]:
Correlation = sum_updata.corr()
Correlation['pm25'].sort_values()

precipcover        -0.439407
tempmin            -0.425032
dew                -0.421254
visibility         -0.406795
feelslikemin       -0.406487
humidity           -0.396195
windgust           -0.394887
precipprob         -0.392298
feelslike          -0.376919
feelslikemax       -0.356599
temp               -0.337350
precip             -0.311866
date               -0.282206
windspeed          -0.262553
cloudcover         -0.252840
tempmax            -0.251645
winddir            -0.002576
moonphase           0.060451
solarenergy         0.070762
uvindex             0.071482
solarradiation      0.071551
sealevelpressure    0.415989
pm25                1.000000
snow                     NaN
snowdepth                NaN
Name: pm25, dtype: float64

In [ ]:
df_weather = df_weather.drop(labels=['feelslikemax','temp','precipprob','windspeed','cloudcover','precip','tempmax','uvindex','solarradiation','solarenergy','winddir','moonphase','snow','snowdepth'], axis=1)

In [ ]:
df_weather

,date,tempmin,feelslikemin,feelslike,dew,humidity,precipcover,windgust,sealevelpressure,visibility
0,1609459200000,3.8,0.6,7.8,-10.9,24.6,0.0,29.7,1027.6,13.5
1,1609545600000,4.8,3.2,10.8,-4.1,39.1,0.0,25.2,1023.8,13.2
2,1609632000000,5.8,4.7,13.8,0.0,42.1,0.0,31.6,1023.1,12.9
3,1609718400000,14.3,14.3,17.1,0.6,33.2,0.0,22.1,1022.0,13.5
4,1609804800000,15.1,15.1,18.3,3.3,37.5,0.0,25.0,1021.0,13.4
...,...,...,...,...,...,...,...,...,...,...
360,1672099200000,9.2,8.6,15.0,6.6,59.6,0.0,21.2,1022.7,13.3
361,1672185600000,12.6,12.6,15.6,5.6,52.0,0.0,35.2,1023.6,13.3
362,1672272000000,10.9,10.9,13.8,4.5,53.8,0.0,35.2,1026.0,13.2
363,1672358400000,8.0,5.5,11.2,4.9,63.3,0.0,36.0,1027.1,13.2


In [ ]:
weather_fg = fs.get_or_create_feature_group(
        name = 'weather_fg_clean',
        description = 'Weather characteristics of each day after cleaning',
        version = 1,
        primary_key = ['date'],
        online_enabled = True,
        event_time = 'date'
    )    

weather_fg.insert(df_weather)